# **NEAREST NEIGHBOR SEARCH FOR TEXT DOCUMENTS WITH LSH**



---

In this notebook we will implement Locality Sensitive Hashing to detect pairs of duplicates in the dataset of Computer product descriptions from Amazon.

We will use a family of Cryptographic hashing functions and we will compare the results obtained with the results of the pairwise comparison on shingles. Also, the running time required to find duplicate pairs will be compared.


In [ ]:
import re
import pandas as pd
import hashlib
import numpy as np
import itertools
import time

## Importing the product dataset

In [ ]:
df = pd.read_csv(r'computer.tsv', sep='\t' )
pd.set_option('display.max_colwidth', -1)

print(df.shape)
df.head(10)

(306, 5)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Description,Price,Prime,Url,Ratings
0,"Computer portatile da 15,6 pollici, CPU Intel N3350, Windows 10 Pro OS,4 GB RAM 64 GB SSD, Full HD 1920 x 1080","298,00 €",1,/Computer-portatile-pollici-Intel-Windows/dp/B08KFZNGGH/ref=sr_1_1?dchild=1&keywords=computer&qid=1604664192&sr=8-1,-1
1,PC DELL 7010 SFF Intel Core i5 3470 3.20Ghz/RAM 8GB/500GB/DVD+RW/WIN 10 PRO (Ricondizionato),"167,99 €",0,/DELL-Intel-3-20Ghz-500GB-Ricondizionato/dp/B07RK9N3WM/ref=sr_1_2?dchild=1&keywords=computer&qid=1604664192&sr=8-2,"4,3"
2,"Laptop da 14 pollici (Intel Celeron a 64 bit, 4 GB di RAM DDR3, 64 GB di eMMC, batteria da 10000 Mah, webcam HD, sistema operativo Windows 10 preinstallato, display IPS 1366 * 768 FHD) Notebook","254,99 €",1,/pollici-batteria-operativo-preinstallato-Notebook/dp/B08BHRW3KB/ref=sr_1_3?dchild=1&keywords=computer&qid=1604664192&sr=8-3,"2,2"
3,"HUAWEI MateBook 14 Laptop, Ultrabook DisplayFullView 2K , AMD Ryzen 7 4800H, 16GB RAM, SSD da 512GB PCIe , Fingerabdrucksensor, Huawei Share, Windows 10 Home, Space Gray, KelvinL-WFE9B","1.099,00 €",1,/HUAWEI-Ultrabook-DisplayFullView-Fingerabdrucksensor-KelvinL-WFE9B/dp/B08J83G2M8/ref=sr_1_4?dchild=1&keywords=computer&qid=1604664192&sr=8-4,"4,3"
4,"Acer Chromebook 314 CB314-1H-C2W1 Notebook, Pc Portatile con Processore Intel Celeron N4000, Ram 4GB DDR4, eMMC 64 GB, Display 14"" Full HD LED LCD, Scheda Grafica Intel, Chrome OS, Silver, [CB]","349,00 €",1,/Acer-Chromebook-CB314-1H-C2W1-Portatile-Processore/dp/B0855696F5/ref=sr_1_5?dchild=1&keywords=computer&qid=1604664192&sr=8-5,"4,3"
5,"Pc portatile 14 Pollici TECLAST F7S Computer Portatile Intel Celeron N3350,128 GB SSD, 8 GB RAM, 4K, 1920 x 1080, Windows 10 Grigio","294,99 €",1,/portatile-TECLAST-F7S-Computer-Portatile/dp/B089QDVXYB/ref=sr_1_6?dchild=1&keywords=computer&qid=1604664192&sr=8-6,"3,7"
6,"Lenovo IdeaPad Slim 1 Notebook, Display 14"" HD, Processore AMD A4-9120e, 64 GB eMMC 5.1, RAM 4 GB, Windows 10, Platinum Grey",0 €,0,/Lenovo-Notebook-Processore-A4-9120e-Platinum/dp/B089NXQXWD/ref=sr_1_7?dchild=1&keywords=computer&qid=1604664192&sr=8-7,"4,1"
7,"HP – PC 15s-eq0021nl Notebook, AMD Ryzen 3, RAM 8 GB, SSD 256 GB, Grafica AMD Radeon Vega 3, Windows 10 Home S, Schermo 15” FHD SVA Antiriflesso, Micro-Edge, Fast Charge, Lettore SD/Micro SD, Argento",0 €,0,/HP-eq0021nl-Notebook-Antiriflesso-Micro-Edge/dp/B082YMS5Q1/ref=sr_1_8?dchild=1&keywords=computer&qid=1604664192&sr=8-8,"4,4"
8,"ACEPC AK1 Mini PC,Windows 10(64 bit) Computer desktop Intel Celeron Apollo Lake J3455 (fino a 2,3GHz) Computer desktop[8GB/120GB/Supporto SSD da 2,5""/ SSD mSATA/Doppio WiFi/Gigabit Ethernet/BT 4.2/4K]","249,90 €",1,/ACEPC-AK1-Computer-Supporto-Ethernet/dp/B07SKGGGP7/ref=sr_1_9?dchild=1&keywords=computer&qid=1604664192&sr=8-9,"4,4"
9,"Lenovo IdeaPad Flex 5 Notebook Convertibile, Display 14"" Full HD Touch, Processore Intel Core i5-1035G1, 512 GB SSD, RAM 8 GB, Fingerprint, Lenovo Digital Pen, Windows 10, Graphite Grey","899,99 €",1,/Lenovo-Convertibile-Processore-i5-1035G1-Fingerprint/dp/B089NWXD2L/ref=sr_1_10?dchild=1&keywords=computer&qid=1604664192&sr=8-10,"4,2"


## Implementing the classes and the methods



**SHINGLING**
1. Implement a class that, given a document, creates its set of character shingles of some length k.
Then represent the document as the set of the hashes of the shingles, for some hash function.

In [ ]:
class ShinglingClass():


  # This function creates shingles of length k
  def shingling(self,text, k):

    #Some preprocessing
    text = text.lower()
    text = re.sub(r'[^\w]', ' ', text) #substitute symbols with spaces
    text = re.sub("  ", " ",text) #remove multiple spaces (can be generated by previous line)
    #We decide to keep the spaces in the shingling, to decrease the false positives,
    #for example, we want to distinguish between 'plane has touch down' and 'threw a touchdown'.

    # WORD-LEVEL SHINGLING-----------------------------------------------------
    #tokens = text.split()
    #return [' '.join(tokens[i:i+k])
    #                 for i in range(len(tokens) - k + 1)]

    #CHARACTER-LEVEL SHINGLING
    s_list = [text[i:i + k] for i in range(len(text) - k + 1)]
    return set(s_list)



  # This function creates the hash of the shingles
  def hash_shingles(self,shingles):
    hashed_shingles = []
    for shingle in shingles:
      hashed_shingles.append(hash(shingle))
    return hashed_shingles


#-------------------------------------------------------------------------------
#How it works: this is an example
x = "Ciao0  ciao.0"
shing = ShinglingClass()
print(x)
print(len(x))
sh = shing.shingling(x,5)
print(len(sh))
print(sh)
print(shing.hash_shingles(sh))

  

Ciao0  ciao.0
13
8
{'ciao ', ' ciao', '0 cia', 'iao 0', 'ciao0', 'iao0 ', 'ao0 c', 'o0 ci'}
[-5738604203726370592, 8366194963876463625, 5170184148546514892, -1805593592384820244, 2886255221790268759, -8608254069412878832, 1551742201392623799, -3858830786513922466]
8


**MIN-HASHING**
2. Implement a class, that given a collection of sets of objects (e.g., strings, or numbers), creates
a minwise hashing based signature for each set.

First, we need to define a **hash family** to simulate the n random permutations of the rows of the characteristic matrix.

In [ ]:
# Implement a family of hash functions. It hashes strings and takes an
# integer to define the member of the family.
# Return a hash function parametrized by i

def hashFamily(i):
  resultSize = 8 # how many bytes we want back
  maxLen = 20 # how long can our i be (in decimal)
  salt = str(i).zfill(maxLen)[-maxLen:]
  def hashMember(x):
    return hashlib.sha1((x + salt).encode()).digest()[-resultSize:]
  return hashMember


#-----------------------------------------------------------------------------------
 #h_i = hashFamily(i) is the i-th hash function of the family
 #Calling h_i('0') you create the hash of the string '0'
hh = hashFamily(2)
print(hh('0'), hh('1'))

#.digest() returns a binary output => for example b'WT\x11\xea8-\xe7\xd8'
#.hexdigest() returns an hexadecimal output => for example 382de7d8

b'C\xa5\x001M#[<' b'\xcfN\x0f4;\xdfC\x8e'


Now we implement the class to compute the characteristic matrix and builds the correspondent **signature matrix**, using n hash functions. 

In [ ]:
#sets should be a 2d list
class MinHashingClass():

  def __init__(self, sets): 
        self.sets = sets #instance variable unique to each instance of MinHashingClass
        self.universal_set = [] #set of all distinct shingles (string or hashed format)

  def to_characteristic_matrix(self):
    
    #Define the universal set from the collection
    for set in self.sets:
      for shingle in set:
        if (shingle not in self.universal_set):
          self.universal_set.append(shingle)
    

    #Translate the list of sets into a boolean characteristic matrix:
    # M(i,j)=1 if shingle i is contained in set j

    #Initialization of the characteristic matrix
    n_sets = len(self.sets)
    n_shingles = len(self.universal_set)
    M = np.zeros(shape=(n_shingles,n_sets))
    #Buiding the matrix
    for row in range(0,n_shingles):
      for col in range(0,n_sets):
        if (self.universal_set[row] in self.sets[col]):
          M[row][col] = 1
    return M
  
  
  def to_signature_matrix(self,M,n):
    #n is the number of taken hash functions of the family

    #dimensions of M:
    n_rows = len(M)
    n_cols = len(M[0])

    #Initialize all entries to infinite
    sig = np.full((n, n_cols), b'')
    h = np.full(n, b'')

    for r in range(0,n_rows):
      for i in range(0,n):
        hashf = hashFamily(i)
        h[i] = hashf(str(r))

      for c in range(n_cols):
        if ( M[r][c]==1 ):
          for i in range(0,n):
            if (np.array_equal(sig[i][c], b'')):
              sig[i][c] = h[i]
            if ( sig[i][c] < h[i] ):
              sig[i][c] = sig[i][c]
            else:
              sig[i][c] = h[i]
    return sig



**LOCALITY SENSITIVE HASHING**
3. Implement a class that implements the locally sensitive hashing (LSH) technique, so that,
given a collection of minwise hash signatures of a set of documents, it nds the all the
documents pairs that are near each other.

The **similarity on the signature matrix** is defined as the fraction of the hash functions in which the two signatures agree, so it is the number of rows with the same value divided by the number of total rows.

In [ ]:
def signature_similarity(sig1,sig2):
  n = len(sig1)
  intersection = 0

  for i in range(n):
    if (sig1[i]==sig2[i]):
      intersection += 1
  
  return intersection/n


LSH class to find nearest duplicates:

In [ ]:
class LSHClass():

  def __init__(self, s, b): 
    self.s = s #similarity threshold
    self.b = b #number of bands
    self.candidates = []
    #self.similarity_scores = []


  def get_candidates(self, sig):
    #returns a list of duplicate docs through their IDs

    n_cols = len(sig[0]) #a row has n_cols columns
    n_rows = len(sig) 
    candidates = [] #pairs of docs ending up in the same bucket for at least 1 band
    duplicates = [] #candidates having similarity score > s
    sim_scores = [] #similarity scores of the detected duplicates

    #Split the signature matrix into b bands:
    bands = np.split(sig, self.b) #bands is a list of numpy arrays

    #hash_table is a data structure that allows to store key (hash) value (column=>set=>doc) pairs
    #If two cols agree on all rows of a band, they end up in the same bucket for that band
    hash_table = [] #list of bucket dictionaries 
    # we need a different bucket dictionary for each band so columns with the same vector
    #   in different bands will not hash to the same bucket.
    for band in bands:
      band_buckets = dict()
      for col in range(n_cols):
        col_portion = band[:, col]
        col_portion = hash(col_portion.tostring())
        # we store the index of the column in the key correspondent to the hash 
        #   of the portion of the column in that band
        band_buckets.setdefault(col_portion,[]).append(col) 
      hash_table.append(band_buckets)

    #Check for candidate pairs
    for band_buckets in hash_table:
      for k,v in band_buckets.items():
        if ( len(band_buckets[k]) > 1 ):
          #create pairs between elements
          for pair in itertools.combinations(v,2):
            candidates.append(pair)

    self.candidates = set(candidates)
    return set(candidates)


  def post_filter_candidates(self):

    #Now we take the candidates and we check if their similarity is >= threshold s
    filtered_candidates = []
    for pair in self.candidates:
      index_col1 = pair[0]
      index_col2 = pair[1]
      #Compute the similarity between the two columns in the signature matrix
      col1 = sig[:, index_col1]
      col2 = sig[:, index_col2]
      sim_score = signature_similarity(col1,col2)

      if (sim_score >= self.s):
        filtered_candidates.append(pair)
        #sim_scores.append(sim_score)

    #self.similarity_scores = sim_scores
    
    return filtered_candidates

**SHINGLE JACCARD**
4. To test the LSH algorithm, also implement a class that given the shingles of each of the documents, finds the nearest neighbors by comparing all the shingle sets with each other.

In [ ]:
class NearestNeighbors():
  
  def __init__(self): 
    self.neighbors = [] 
    self.scores = []
    self.n_combinations = 0

  #Computes Jaccard similarity between two sets
  def compute_jaccard (self,A,B):
    A = set(A)
    B = set(B)
    intersection = len(A.intersection(B))
    union = len(A.union(B))
    jaccard = intersection / union

    return jaccard
  
  def find_neighbors(self, sets ):

    scores = []
    neighbors = []
    all_combinations = 0

    #Generate all pairs of columns 
    n_sets = len(sets)
    docIDs = []
    pairs = []

    for i in range(n_sets):
      docIDs.append(i)
    for pair in itertools.combinations(docIDs,2):
      all_combinations += 1
      #Compute jaccard coefficient
      A = sets[pair[0]]
      B = sets[pair[1]]
      jaccard = self.compute_jaccard(A,B)
      if (jaccard >= 0.8):
        scores.append(jaccard)
        neighbors.append(pair)

    self.neighbors = neighbors
    self.scores = scores
    self.n_combinations = all_combinations
  
    return neighbors
    

## Finding Near Duplicates using LSH

In [ ]:
#Choose k as the size of shingles
k=10

shing = ShinglingClass()
print("Applying SHINGLING...")
shingle_df = df['Description'].dropna().apply(lambda z: shing.shingling(z,k))

print(shingle_df.head())
print("\nHASHING shingles...")
shingle_df = df['Description'].dropna().apply(lambda z: shing.hash_shingles(z))

shingle_df.head()

Applying SHINGLING...
0    {50, window,  pollici, , d, full hd, ssd, full ,  gb ram 64, , cpu inte, tel n3350,, a 15,6 pol, gb ssd, fu, 4 gb ssd, , r portatil, windows 10, 5,6 pollic, tile da 15, cpu intel , l n3350, w,  full hd 1, o os,4 gb , 6 pollici,, s 10 pro o, mputer por, 920 x 1080, 1920 x 108, portatile ,  intel n33, ici, cpu i, l hd 1920 , d 1920 x 1, ,4 gb ram , dows 10 pr, ndows 10 p,  gb ssd, f,  da 15,6 p, , windows , indows 10 , b ssd, ful, pro os,4 g, 15,6 polli, pollici, c, rtatile da, llici, cpu, ile da 15,, 0, windows, u intel n3,  cpu intel,  10 pro os, ro os,4 gb, 3350, wind,  portatile,  64 gb ssd,  n3350, wi, ll hd 1920, 0 pro os,4, atile da 1,  pro os,4 ,  hd 1920 x,  windows 1, hd 1920 x , pu intel n, , full hd , omputer po, 64 gb ssd,,  os,4 gb r, puter port, intel n335, 350, windo, le da 15,6, ter portat, i, cpu int, ollici, cp, ortatile d, da 15,6 po,  ram 64 gb, gb ram 64 , sd, full h, tatile da , n3350, win, ci, cpu in, ows 10 pro, lici, cpu , ,6 pollici, 

0    [2327285305615658431, -9062990035501684277, 3874035541517063648, -5266660959964973478, 3871773975480142321, 2307249783634915753, 4329577756000234461, 4295062967841180116, -2089159861098053011, -5266660959964973478, -9062990035501684277, 4295062967841180116, 2307249783634915753, 638239885523082005, 2307249783634915753, -5798693871501297105, 2515775033176219011, 4329577756000234461, -2089159861098053011, -6247273558098350831, 638239885523082005, -2089159861098053011, 3358620583419737909, 164040679829831731, 3991437783730082372, 7518427441921392683, -2089159861098053011, -5266660959964973478, -9062990035501684277, 2515775033176219011, 2515775033176219011, -5798693871501297105, 366257322868641307, -5798693871501297105, 3991437783730082372, -2089159861098053011, 2327285305615658431, 3091337268072463650, -8739729051812113581, -2089159861098053011, -2058404330249021241, 8358974368986303470, 2307249783634915753, 4329577756000234461, 2515775033176219011, -2089159861098053011, 6040220110254

In [ ]:
#Convert df to multidimensional list
sets = shingle_df.values.tolist()
min_hash = MinHashingClass(sets)

#Create the characteristic matrix
print(min_hash.universal_set)
M = min_hash.to_characteristic_matrix()
print("\nThis is the set of all distinct shingles of the collection: ")
print(min_hash.universal_set)
print("\nThis is the characteristic matrix:")
print(M)
print("Shape: ",M.shape)

#Create the signature matrix
print("\nThis is the signature matrix:")
b = 20
r = 13
n = b * r #n is the number of hash functions. Choose b and r, and then n=br
sig = min_hash.to_signature_matrix(M,n) #it tooks a while, since we are using cryptographic hashing functions
print(sig)
print("Shape: ", sig.shape)



[]

This is the set of all distinct shingles of the collection: 
[2327285305615658431, -9062990035501684277, 3874035541517063648, -5266660959964973478, 3871773975480142321, 2307249783634915753, 4329577756000234461, 4295062967841180116, -2089159861098053011, 638239885523082005, -5798693871501297105, 2515775033176219011, -6247273558098350831, 3358620583419737909, 164040679829831731, 3991437783730082372, 7518427441921392683, 366257322868641307, 3091337268072463650, -8739729051812113581, -2058404330249021241, 8358974368986303470, 6040220110254362688, -8622429092379692181, -1106844696282570488, 6409330293419475670, -4499324392711459788, -161503885333630029, -5962147161523952004, -1634362074389154571, -307864657445702325, 8392555058409338455, 8633645249140307856, 3717554918467231849, -3185620147539059690, 6463271443792133288, 3913716859700793612, -764627234758289572, 6168076309469133371, -3669375306141416555, -4561072618482363042, -2279847987130941824, 2239852363235035828, -20171186153120512

In [ ]:

s = 0.8 #They should agree in at least the 80% of the rows
#Plot on wolfram: plot 1-(1-s^r)^b, (s,0,1) such that br=n


start_time = time.time()
lsh = LSHClass(s,b)
duplicate_pairs = lsh.get_candidates(sig)
end_time = time.time()

print("There are ",len(duplicate_pairs), " duplicate pairs.")
print("\n Duplicate pairs returned: ")
print(duplicate_pairs) #pairs of docs having at least r rows (1 band) of signature matrix in common
print("\nTime taken: ", end_time - start_time, " seconds.")

# If you want to post-filter the retrieved pairs
#filtered_duplicates = lsh.post_filter_candidates()
#print("\nThere are ", len(filtered_duplicates), " pairs having similarity >= 0.8.")

There are  1808  duplicate pairs.

 Duplicate pairs returned: 
{(140, 226), (19, 304), (98, 203), (116, 158), (7, 25), (10, 271), (27, 122), (196, 296), (10, 97), (36, 256), (114, 173), (90, 130), (38, 119), (21, 37), (131, 275), (111, 232), (25, 162), (29, 37), (3, 288), (131, 160), (6, 287), (102, 222), (11, 160), (125, 194), (142, 288), (172, 296), (83, 255), (3, 11), (24, 132), (131, 280), (209, 265), (63, 114), (156, 304), (29, 62), (11, 72), (88, 272), (102, 209), (177, 238), (103, 104), (197, 227), (238, 298), (37, 63), (77, 88), (208, 265), (6, 92), (10, 19), (102, 160), (15, 157), (0, 5), (134, 176), (67, 239), (145, 220), (97, 238), (98, 257), (122, 222), (123, 196), (3, 302), (117, 148), (9, 59), (119, 176), (157, 296), (124, 234), (198, 234), (77, 81), (227, 243), (214, 298), (162, 177), (114, 214), (39, 288), (12, 262), (33, 68), (159, 177), (166, 173), (115, 141), (23, 300), (81, 304), (3, 279), (149, 220), (30, 49), (196, 268), (125, 237), (54, 171), (110, 259), (275, 29

Visualizing some of the results in Pandas:

In [ ]:

descriptions1 = []
descriptions2 = []

for pair in list(duplicate_pairs):
  descriptions1.append(df['Description'].iloc[pair[0]])
  descriptions2.append(df['Description'].iloc[pair[1]])

duplicates_df = pd.DataFrame(
    {'pair': list(duplicate_pairs),
     'document1': descriptions1,
     'document2': descriptions2
    })


duplicates_df.head(76)

,pair,document1,document2
0,"(140, 226)","AITOO Webcam HD 1080p con microfono, Full HD 1080P Webcam USB Pc Computer Camera con Microfono Driver Video Webcam per Insegnare Online Live Broadcast","KROSER Borsa del Portatile Cartella Leggera Espandibile Aggiornata da 18"" per Laptop 17.3"" Borsa da Lavoro Premium Borsa Messenger Idrorepellente con Tasche RFID per Scuola/Viaggio/Donna/Uomo-Nero"
1,"(19, 304)",PC Computer Desktop Gaming Entry Level Fujitsu Esprimo P710 Windows 10 Professional Intel Core i5-3470S Memoria Ram 8GB DDR3 Hard Disk 500GB DVD-ROM USB 3.0 GeForce GT710 2GB (Ricondizionato),"Lenovo IdeaCentre AIO A340-24IWL Computer desktop All-in-One con Processore Intel Core i5-8265U, 8 GB di RAM, 128 GB SSD, 1 TB HDD, masterizzatore DVD, Intel UHD, Windows 10 Home 64, nero"
2,"(98, 203)","VistaProtect – Filtro Anti Luce Blu Premium per Schermo Laptop PC, Rimovibile (13.3"" Pollici)","Elekin Supporto per Computer Portatile,Supporto per Laptop Ventilato in Alluminio,Supporto PC Portatile 6 Livelli Angolazione Regolabile Pieghevole Supporto per PC/iPad/Notebook/Tablet,10-17"""
3,"(116, 158)","Logitech M185 Mouse Wireless, 2.4 GHz con Mini Ricevitore USB, Durata Batteria Fino a 12 Mesi, Rilevamento Ottico 1000 DPI, Ambidestro, PC/Mac/Laptop, Grigio","Acer Nitro 5 AN515-54-55CQ Notebook Gaming, Intel Core i5-9300H, Ram 8 GB, 512 GB PCIe NVMe SSD, Display 15.6"" FHD IPS 60 Hz LCD, Nvidia GeForce GTX 1650 4 GB GDDR5, No Sistema Operativo"
4,"(7, 25)","HP – PC 15s-eq0021nl Notebook, AMD Ryzen 3, RAM 8 GB, SSD 256 GB, Grafica AMD Radeon Vega 3, Windows 10 Home S, Schermo 15” FHD SVA Antiriflesso, Micro-Edge, Fast Charge, Lettore SD/Micro SD, Argento","HP-PC Pavilion 14-ce3034nl Notebook, Intel Core i5-1035G1, RAM 8 GB, SSD 512 GB, Grafica UHD Intel, Windows 10 Home, Schermo 14"" FHD Antiriflesso, Lettore Impronte Digitali, Lettore Micro SD, Argento"
...,...,...,...
71,"(159, 177)","Lenovo ThinkBook 15 Grigio Computer Portatile 39,6 cm (15.6"") 1920 x 1080 Pixel Intel® Core i7 di Decima Generazione 16 GB DDR4-SDRAM 512 GB SSD Wi-Fi 6 (802.11ax) ThinkBook 15, Intel®","Acer Aspire 3 A315-56-582U Computer Portatile Nero 39,6 cm (15.6"") 1920 x 1080 Pixel Intel® Core i5 di Decima Generazione 8 GB DDR4-SDRAM 1000 GB SSD Wi-Fi 5 (802.11ac) Windows 10 Home"
72,"(166, 173)","Beelink BT3PRO II Mini PC, Mini Computer Desktop con HDMI e VGA, Processore Intel Atom X5-Z8350, WiFi da 4GB+64GB, 2,4G/5,8G Dual Band, BT 4.0, 4K, 1000 Mbps LAN, Supporto Windows 10","Acer Aspire C24-320 Computer Fisso All in One, Pc Desktop con Processore AMD A9 A9-9425, Ram 8 GB DDR4, 256 GB M.2 SATA SSD, Display 23.8"" FHD IPS, AMD Radeon R5, Wireless Lan, Windows 10 Home"
73,"(115, 141)","Lenovo V V15 Grigio Computer portatile 39,6 cm [15.6""] 1920 x 1080 Pixel Intel® Core™ i3 di ottava generazione 8 GB DDR4-SDRAM 256 GB SSD Wi-Fi 5","DELL Latitude 5500 Nero Computer portatile 39,6 cm (15.6"") 1920 x 1080 Pixel Intel® Core™ i7 di ottava generazione i7-8665U 16 GB DDR4-SDRAM 512 GB SSD"
74,"(23, 300)","HP - PC Pavilion 15-cw1085nl Notebook, AMD Ryzen 5 3500U, RAM 8 GB, SSD 512 GB, Grafica AMD Radeon Vega 8, Windows 10 Home, Schermo 15.6"" FHD IPS, Lettore Micro SD, HDMI, USB-C, RJ-45, Webcam, Argento","HP - PC Pavilion 15-eh0000sl Notebook, AMD Ryzen 5 4500U, RAM 8 GB, SSD 512 GB, Grafica AMD Graphics, Windows 10 Home, Schermo 15.6"" FHD IPS, Lettore Impronte digitali, Webcam, Argento"


## Finding duplicates comparing the all pairs of shingle sets

In [ ]:
nn = NearestNeighbors()

print("Finding duplicate pairs...")
start_time = time.time()
neighbor_pairs = nn.find_neighbors(sets)
end_time = time.time()

n_combinations = nn.n_combinations
print("Computing similarity between ", n_combinations, " pairs...\n")
print("Time taken: ", end_time-start_time, " seconds.")
print("There are ", len(neighbor_pairs), " duplicates.")
print("\nDuplicate pairs returned: ")
print(neighbor_pairs)

Finding duplicate pairs...
Computing similarity between  46665  pairs...

Time taken:  0.8941807746887207  seconds.
There are  526  duplicates.

Duplicate pairs returned: 
[(0, 173), (0, 177), (1, 30), (1, 34), (1, 96), (2, 151), (2, 236), (3, 10), (3, 11), (3, 124), (3, 134), (3, 199), (3, 208), (3, 230), (3, 265), (3, 288), (3, 300), (4, 21), (4, 37), (4, 63), (4, 72), (4, 171), (6, 11), (6, 20), (6, 173), (6, 186), (6, 300), (9, 20), (9, 25), (9, 76), (9, 81), (9, 114), (9, 151), (9, 199), (9, 230), (9, 236), (9, 274), (9, 300), (9, 301), (10, 23), (10, 46), (10, 134), (10, 186), (10, 199), (10, 208), (10, 230), (10, 262), (10, 265), (10, 275), (10, 285), (10, 288), (10, 300), (11, 23), (11, 134), (11, 158), (11, 163), (11, 208), (11, 300), (12, 124), (15, 24), (15, 77), (15, 123), (15, 157), (15, 177), (15, 216), (15, 268), (16, 302), (17, 199), (17, 265), (18, 265), (19, 29), (19, 62), (19, 97), (19, 103), (19, 198), (19, 234), (21, 63), (21, 171), (21, 300), (22, 177), (23, 25), 

In [ ]:
sim_scores_all = nn.scores
descriptions1 = []
descriptions2 = []

for pair in neighbor_pairs:
  descriptions1.append(df['Description'].iloc[pair[0]])
  descriptions2.append(df['Description'].iloc[pair[1]])

duplicates_df = pd.DataFrame(
    {'pair': neighbor_pairs,
     'jaccard_coeff': sim_scores_all,
     'document1': descriptions1,
     'document2': descriptions2
    })

duplicates_df = duplicates_df.sort_values(by=['jaccard_coeff'], ascending=False)
duplicates_df.head(106)
#duplicates_df.tail(10)

,pair,jaccard_coeff,document1,document2
236,"(89, 165)",1.000000,"Apple MacBook Air Argento Computer Portatile 33,8 cm (13.3"") 2560 x 1600 Pixel Intel® Core i3 di Decima Generazione 8 GB LPDDR4x-SDRAM 256 GB SSD Wi-Fi 5 (802.11ac) macOS Catalina MacBook","Apple MacBook PRO Grigio Computer Portatile 33,8 cm (13.3"") 2560 x 1600 Pixel Intel® Core i5 di Decima Generazione 16 GB LPDDR4x-SDRAM 512 GB SSD Wi-Fi 5 (802.11ac) macOS Catalina MacBook"
467,"(203, 254)",1.000000,"Elekin Supporto per Computer Portatile,Supporto per Laptop Ventilato in Alluminio,Supporto PC Portatile 6 Livelli Angolazione Regolabile Pieghevole Supporto per PC/iPad/Notebook/Tablet,10-17""","Elekin Supporto per Computer Portatile,Supporto per Laptop Ventilato in Alluminio,Supporto PC Portatile 6 Livelli Angolazione Regolabile Pieghevole Supporto per PC/iPad/Notebook/Tablet,10-17"""
150,"(42, 55)",1.000000,Virtual Revolution,Virtual Revolution
179,"(62, 234)",1.000000,"PC Computer Desktop HP Elite 8300, Windows 10 Professional, Intel Core i7-3770, Memoria Ram 8GB DDR3, SSD 120GB, DVD-ROM, USB 3.0 (Ricondizionato)","PC Computer Desktop HP Elite 8300, Windows 10 Professional, Intel Core i7-3770, Memoria Ram 8GB DDR3, SSD 120GB, DVD-ROM, USB 3.0 (Ricondizionato)"
180,"(66, 224)",1.000000,"Lettore Masterizzatore Blu Ray Dvd, USB 3.0 Esterno Blu-Ray Portatile 3D CD Dvd RW Lettore Disco per Laptop/Desktop/MacBook, Win 7/8/10, Linux, PC","Lettore Masterizzatore Blu Ray Dvd, USB 3.0 Esterno Blu-Ray Portatile 3D CD Dvd RW Lettore Disco per Laptop/Desktop/MacBook, Win 7/8/10, Linux, PC"
...,...,...,...,...
248,"(97, 103)",0.918367,"PC Computer Desktop Fujitsu Esprimo P720, Windows 10 Professional, Intel Core i5-4570S, Memoria Ram 8GB DDR3, SSD 240GB + HDD 500GB, DVD-ROM (Ricondizionato)","PC Computer Desktop Tower HP Elite 8200, Windows 10 Professional, Intel Core i5-2400, Memoria Ram 8GB DDR3, SSD 120GB + HDD 500GB, DVD-ROM (Ricondizionato)"
129,"(36, 166)",0.909091,"Beelink BT3 Pro II Mini PC Support Windows 10, Intel Atom x5-Z8350 Processor 4GB Ram 64GB Rom Graphics HD 400, 1000Mbps LAN MINI Computer con 5.8G+2.4G WiFi HDMI+VGA Outputs/Bluetooth 4.0/USB 3.0","Beelink BT3PRO II Mini PC, Mini Computer Desktop con HDMI e VGA, Processore Intel Atom X5-Z8350, WiFi da 4GB+64GB, 2,4G/5,8G Dual Band, BT 4.0, 4K, 1000 Mbps LAN, Supporto Windows 10"
482,"(216, 268)",0.909091,"msi Gaming GS66 10SFS-056 Stealth Nero Computer Portatile 39,6 cm (15.6"") 1920 x 1080 Pixel Intel® Core i9 di Decima Generazione 16 GB DDR4-SDRAM 1000 GB SSD NVIDIA GeForce RTX 2070 Super Wi-Fi 6","msi Gaming GE75 10SFS-067 Raider Alluminio, Nero Computer Portatile 43,9 cm (17.3"") 1920 x 1080 Pixel Intel® Core i9 di Decima Generazione 16 GB DDR4-SDRAM 1512 GB HDD+SSD NVIDIA GeForce RTX 2070"
103,"(25, 300)",0.901961,"HP-PC Pavilion 14-ce3034nl Notebook, Intel Core i5-1035G1, RAM 8 GB, SSD 512 GB, Grafica UHD Intel, Windows 10 Home, Schermo 14"" FHD Antiriflesso, Lettore Impronte Digitali, Lettore Micro SD, Argento","HP - PC Pavilion 15-eh0000sl Notebook, AMD Ryzen 5 4500U, RAM 8 GB, SSD 512 GB, Grafica AMD Graphics, Windows 10 Home, Schermo 15.6"" FHD IPS, Lettore Impronte digitali, Webcam, Argento"


## Comparing results

- **False positives** are docs that are detected as duplicates by LSH but not comparing all pairs;
- **False negatives** are docs that are not detected as duplicated by LSH, but are detected comparing all pairs.

We try to balance the presence of false positives and false negatives, but giving higher importance to the minimization of false negatives (because we can always post filter results to remove false positives, checking the similarity on the signatures, while false negatives can not be retrieved in any way using LSH).


In this case precision and fscore are not really good metrics, since they do not take into account the false negatives. For example, having 4000 false positives is not a huge problem compared to the number of all the combinations of pairs.

In [ ]:
true_positives = []
false_negatives = []

#neighbor_pairs are the result of computing jaccard between every pair
#duplicate_pairs are the result of LSH

for pair in neighbor_pairs:
  if pair in duplicate_pairs:
    true_positives.append(pair)
  else:
    false_negatives.append(pair)

TP = len(true_positives) #true positives
FN = len(false_negatives) #false negatives
FP = len(duplicate_pairs)-TP #false positives
TN = n_combinations - FP - FN - TP #true negatives

recall = TP / (TP + FN)
precision = TP / (TP+FP)
f_score = 2 * precision * recall / (precision + recall)
accuracy = (TP+TN) / (n_combinations)

print("Size of the intersection (true positives): ", TP )
print("Size of false negatives: ", FN )
print("Size of false positives: ", FP)
print("Size of true negatives: ", TN)

print("\nPrecision = ", precision)
print("Recall = ", recall)
print("F1-Score = ", f_score)
print("Accuracy = ", accuracy)

Size of the intersection (true positives):  428
Size of false negatives:  98
Size of false positives:  1380
Size of true negatives:  44759

Precision =  0.23672566371681417
Recall =  0.8136882129277566
F1-Score =  0.3667523564695801
Accuracy =  0.9683274402657238


In [ ]:
print((true_positives))

[(0, 36), (0, 68), (1, 194), (3, 46), (3, 53), (3, 67), (3, 86), (3, 132), (3, 169), (4, 67), (4, 183), (5, 6), (5, 14), (5, 18), (5, 25), (5, 26), (5, 33), (5, 68), (5, 83), (5, 88), (5, 95), (5, 101), (5, 113), (5, 119), (5, 172), (5, 237), (6, 12), (6, 15), (6, 22), (6, 30), (6, 42), (6, 78), (7, 11), (7, 145), (8, 59), (8, 67), (10, 194), (11, 62), (11, 83), (11, 101), (11, 113), (11, 119), (11, 145), (11, 181), (12, 30), (13, 81), (13, 155), (14, 26), (14, 33), (14, 45), (14, 47), (14, 61), (14, 62), (14, 83), (14, 87), (14, 88), (14, 93), (14, 95), (14, 113), (14, 135), (14, 181), (14, 223), (15, 24), (15, 56), (15, 101), (16, 110), (16, 168), (17, 45), (17, 49), (17, 51), (17, 62), (17, 93), (17, 118), (17, 135), (17, 222), (18, 26), (18, 61), (18, 113), (19, 95), (19, 119), (19, 194), (19, 201), (19, 237), (20, 94), (22, 172), (22, 201), (23, 143), (24, 32), (24, 56), (24, 78), (24, 181), (24, 194), (25, 43), (25, 84), (25, 95), (25, 172), (25, 201), (26, 33), (26, 83), (26, 10